In [1]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np, pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [2]:
def getMongoUri(auth_filename='auths/mongo_uri.pkl'):
    with open(auth_filename, 'rb') as file:
        pwd = pickle.load(file)
    return pwd['mongo_uri']

def createMongoClient(db='aita'):
    mongo_uri = getMongoUri()
    client = MongoClient(mongo_uri)
    db = client.aita
    return client, db

In [6]:
client, db = createMongoClient()

In [86]:
query_string = 'password'
post_id = 'aeeomg'
status = 1

In [72]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [87]:
qvector = embed([query_string]).numpy()

In [102]:
rids = db.queries.find_one({"query": query_string}, projection={'reddit_ids':1})['reddit_ids']
feedbackPost = db.clean.posts.find_one({"id": post_id}, projection={'vector':1})

In [103]:
feedbackVector = np.array(feedbackPost['vector'])

In [90]:
SENSITIVITY = 0.75
def update_embedding(q_vector, feedback_vector, feedback):
    new_q = (1 - feedback*SENSITIVITY)*q_vector + feedback*SENSITIVITY*feedback_vector
    return new_q

In [91]:
qprime = update_embedding(qvector, feedbackVector, status)

In [92]:
postSubset = []
for p in db.clean.posts.find({'id': {'$in':rids}}, projection={'id':1, 'vector':1}):
    postSubset.append(p)

In [93]:
vectors = np.array([x['vector'] for x in postSubset])

In [94]:
vectors.shape

(2279, 512)

In [95]:
scores = qprime.dot(vectors.T).flatten()

In [96]:
ret = []
for i in range(len(postSubset)):
    id_ = postSubset[i]['id']
    score = scores[i]
    ret.append((id_,score))

In [97]:
ret.sort(key=lambda x:x[1], reverse=True)

In [98]:
db.queries.update_one({'query':query_string}, {'$set': { "ranked_post_ids" : ret}});

In [99]:
ret

[('aeeomg', 0.7557883383362038),
 ('aeev8b', 0.6878611907993746),
 ('cpi8kt', 0.3996116226409548),
 ('cd18t1', 0.395251056865375),
 ('cy3a30', 0.39248240660672895),
 ('dl8i56', 0.3918325443049702),
 ('cplm4j', 0.3909553832431958),
 ('gqabcx', 0.38578303415387644),
 ('i3wpic', 0.38565841666738787),
 ('e1my2y', 0.3812343341221519),
 ('cdf6fb', 0.38075583092196436),
 ('bakvq8', 0.3803207893102121),
 ('fva7id', 0.3788162574560665),
 ('cd178c', 0.3780727009101086),
 ('f4k5hr', 0.377170250336078),
 ('dgy64k', 0.37336920232628135),
 ('efb0d6', 0.37005566487843666),
 ('e4wxbo', 0.3686995583733861),
 ('dhcsq4', 0.36745185635490685),
 ('dy822e', 0.3660363892391626),
 ('b0pgs7', 0.36571527128786546),
 ('buzq4w', 0.36540330533283055),
 ('diboyw', 0.3650286268801851),
 ('d5m306', 0.3614880852543416),
 ('htqvvl', 0.3590994096412678),
 ('acs3nw', 0.3587306012141768),
 ('ftgyxa', 0.3582190355720666),
 ('atci9a', 0.35731309064875627),
 ('axwr84', 0.3568785522236272),
 ('clynle', 0.356450459934271),
 ('